<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Generate and Download Image from spreadsheet
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Google+Sheets+-+Generate+and+Download+Image+from+spreadsheet:+Error+short+description">Bug report</a>

**Tags:** #googlesheet #gsheet #midjourney #naas_drivers #imaginedev

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-11-13 (Created: 2023-11-13)

**Description:** This notebook will generate images and download them in your local from a Google Sheets spreadsheet.

**References:**
- [ImagineAPI.dev Documentation](https://docs.imagineapi.dev/aypeeeye/image)

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet
import urllib.parse
import requests
import os
import naas
import pydash as py
import time
from IPython.display import Image, display

### Setup variables
- `spreadsheet_url`: This variable stores the URL of a Google Sheets spreadsheet.
- `sheet_name`: This variable stores the name of a specific sheet within the Google Sheets spreadsheet.
- `api_key`: API key provided by ImagineAPI.dev. [Get your API key](https://docs.imagineapi.dev/aypeeeye/authentication)
- `imagineapi_url`: Your ImagineAPI URL
- `output_dir`: This variable stores the output directory to store images.

In [ ]:
spreadsheet_url = "https://docs.google.com/spreadsheets/d/"
sheet_name = "Ref"
col_image_name = "Character"
col_image_url = "Avatar"

api_key = naas.secret.get("IMAGINEAPI_API_KEY")
imagineapi_url = "https://cl-151.imagineapi.dev/admin/login"
prompt = " profile picture"

output_dir = "../.github/assets/characters"

## Model

### Get image from output directory

In [ ]:
output_dir = "../.github/assets/characters"
os.makedirs(output_dir, exist_ok=True)
images = sorted(os.listdir(output_dir))
print(images)

### Get data from Google Sheets spreadsheet

In [ ]:
df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
print("Row fetched:", len(df_input))
df_input.head(1)

### Generate and Download images

In [ ]:
def create_image(
    api_key,
    base_url,
    prompt
):
    data = {
        "prompt": prompt,
    }
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    domain = base_url.split("https://")[1].split("/")[0]
    url  = f"https://{domain}/items/images/"
    res = requests.post(url, json=data, headers=headers)
    res.raise_for_status
    return py.get(res.json(), "data.id")

def get_image(
    api_key,
    base_url,
    image_id
):
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    domain = base_url.split("https://")[1].split("/")[0]
    url  = f"https://{domain}/items/images/{image_id}"
    res = requests.get(url, headers=headers)
    res.raise_for_status
    return res.json()

def generate_image(
    api_key,
    base_url,
    prompt
):
    # Init
    data = {}
    
    # Create Image
    image_id = create_image(
        api_key,
        base_url,
        prompt
    )
    
    while True:
        # Check status
        data = get_image(
            api_key,
            base_url,
            image_id
        )
        status = py.get(data, "data.status")
        progress = py.get(data, "data.progress")
        if status == "pending":
            progress = 0
        if status == "completed":
            progress = 100
        print(f"- Status: {status}")
        print(f"- Progress: {progress}%")
        print()
        if status in ["completed", "failed"]:
            break
        time.sleep(7)
    return data

def dowload_image(
    image_url, 
    image_path
):
    # Send a GET request to the image URL
    response = requests.get(image_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Get the content of the response
        image_content = response.content

        # Save the image to a file
        with open(image_path, "wb") as file:
            file.write(image_content)
        print("✅ Image downloaded successfully.")
    else:
        print(f"Failed to download image. Error: {response.text}")

for i, row in df_input.iterrows():
    # Init
    image_name = row[col_image_name]
    image_url = row[col_image_url]
    image_name_ext = image_name + ".png"
    image_path = os.path.join(output_dir, image_name_ext)
    print(f"{i+1} - Started with:", image_name)
    
    # Generate Image
    if image_name_ext in images:
        print(f"✅ Image already exists in: {output_dir}")
    elif not str(image_url).startswith("https://") or str(image_url) == "None":
        image_prompt = image_name + prompt
        print(f"Prompt: {image_prompt}")
        image_data = generate_image(api_key, imagineapi_url, image_prompt)
        time.sleep(10)
        urls = py.get(image_data, "data.upscaled_urls")
        for j, url in enumerate(urls):
            print("URL:", url)
            display(Image(url=url, width=400, height=400))
            if j == 0:
                image_name_ext = image_name + ".png"
            else:
                image_name_ext = image_name + f'_{str(j)}' + ".png"
            image_path = os.path.join(output_dir, image_name_ext)
            dowload_image(
                url, 
                image_path
            )
    elif image_name_ext not in images:
        dowload_image(
            image_url, 
            image_path
        )
        
    # Update dataframe
    df_input.loc[i, col_image_url] = f"https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/characters/{urllib.parse.quote(image_name_ext)}"

df_input

## Output

### Send data to Google Sheets

In [ ]:
gsheet.connect(spreadsheet_url).send(data=df_input, sheet_name=sheet_name, append=False)